## Introduction to Databases

### Using [MongoDB](https://docs.mongodb.com/manual/administration/install-community/)

!sudo apt install pymongo

In [1]:
import datetime
import pprint

import pymongo
from pymongo import MongoClient

## Making a Connection with MongoClient

The first step when working with PyMongo is to create a MongoClient to the running mongod instance. Doing so is easy:

In [2]:
client = MongoClient('localhost', 27017)

## Getting/Creating a Database

A single instance of MongoDB can support multiple independent databases. When working with PyMongo you access databases using attribute style access on MongoClient instances:

In [3]:
db = client.test_database
#db = client['test_database']

## Check if Database Exists

In [4]:
print(client.list_database_names())

['admin', 'config', 'local']


In [5]:
dblist = client.list_database_names()
if "test_database" in dblist:
    print("The database exists.")

## Getting a Collection

A collection is a group of documents stored in MongoDB, and can be thought of as roughly the equivalent of a table in a relational database. Getting a collection in PyMongo works the same as getting a database:

In [6]:
collection = db.test_collection
#collection = db['test_collection']

In [7]:
print(db.list_collection_names())

[]


## Documents

Data in MongoDB is represented (and stored) using JSON-style documents. In PyMongo we use dictionaries to represent documents. As an example, the following dictionary might be used to represent a blog post:

In [8]:
post = {"author": "Mike",
        "text": "My first blog post!",
        "tags": ["mongodb", "python", "pymongo"],
        "date": datetime.datetime.utcnow()}

## Inserting a Document

To insert a document into a collection we can use the insert_one() method:

In [9]:
posts = db.posts
post_id = posts.insert_one(post).inserted_id
post_id

ObjectId('5f9ab5edb9603eb43d9407ed')

When a document is inserted a special key, "_id", is automatically added if the document doesn’t already contain an "_id" key. The value of "_id" must be unique across the collection. insert_one() returns an instance of InsertOneResult. For more information on "_id", see the documentation on _id.

After inserting the first document, the posts collection has actually been created on the server. We can verify this by listing all of the collections in our database:

In [10]:
print(client.list_database_names())
print(db.list_collection_names())

['admin', 'config', 'local', 'test_database']
['posts']


## Getting a Single Document With find_one()

The most basic type of query that can be performed in MongoDB is find_one(). This method returns a single document matching a query (or None if there are no matches). It is useful when you know there is only one matching document, or are only interested in the first match. Here we use find_one() to get the first document from the posts collection:

In [11]:
pprint.pprint(posts.find_one())

{'_id': ObjectId('5f9ab5edb9603eb43d9407ed'),
 'author': 'Mike',
 'date': datetime.datetime(2020, 10, 29, 12, 30, 37, 656000),
 'tags': ['mongodb', 'python', 'pymongo'],
 'text': 'My first blog post!'}


The result is a dictionary matching the one that we inserted previously.

Note: The returned document contains an "_id", which was automatically added on insert.

find_one() also supports querying on specific elements that the resulting document must match. To limit our results to a document with author “Mike” we do:

In [12]:
pprint.pprint(posts.find_one({"author": "Mike"}))

{'_id': ObjectId('5f9ab5edb9603eb43d9407ed'),
 'author': 'Mike',
 'date': datetime.datetime(2020, 10, 29, 12, 30, 37, 656000),
 'tags': ['mongodb', 'python', 'pymongo'],
 'text': 'My first blog post!'}


In [13]:
pprint.pprint(posts.find_one({"text": "My first blog post!"}))

{'_id': ObjectId('5f9ab5edb9603eb43d9407ed'),
 'author': 'Mike',
 'date': datetime.datetime(2020, 10, 29, 12, 30, 37, 656000),
 'tags': ['mongodb', 'python', 'pymongo'],
 'text': 'My first blog post!'}


In [14]:
pprint.pprint(posts.find_one({"tags": "mongodb"}))

{'_id': ObjectId('5f9ab5edb9603eb43d9407ed'),
 'author': 'Mike',
 'date': datetime.datetime(2020, 10, 29, 12, 30, 37, 656000),
 'tags': ['mongodb', 'python', 'pymongo'],
 'text': 'My first blog post!'}


In [15]:
posts.find_one({"author": "Eliot"})

## Querying By ObjectId

We can also find a post by its _id, which in our example is an ObjectId:

In [16]:
post_id

ObjectId('5f9ab5edb9603eb43d9407ed')

In [17]:
pprint.pprint(posts.find_one({"_id": post_id}))

{'_id': ObjectId('5f9ab5edb9603eb43d9407ed'),
 'author': 'Mike',
 'date': datetime.datetime(2020, 10, 29, 12, 30, 37, 656000),
 'tags': ['mongodb', 'python', 'pymongo'],
 'text': 'My first blog post!'}


Note that an ObjectId is not the same as its string representation:

In [18]:
posts.find_one({"_id": str(post_id)})

In [19]:
type(post_id)

bson.objectid.ObjectId

A common task in web applications is to get an ObjectId from the request URL and find the matching document. It’s necessary in this case to convert the ObjectId from a string before passing it to find_one:

In [20]:
from bson.objectid import ObjectId

# The web framework gets post_id from the URL and passes it as a string
def get(post_id):
    # Convert from string to ObjectId:
    document = client.db.collection.find_one({'_id': ObjectId(post_id)})

## Bulk Inserts

In order to make querying a little more interesting, let’s insert a few more documents. In addition to inserting a single document, we can also perform bulk insert operations, by passing a list as the first argument to insert_many(). This will insert each document in the list, sending only a single command to the server:

In [21]:
new_posts = [{"author": "Mike",
              "text": "Another post!",
              "tags": ["bulk", "insert"],
              "date": datetime.datetime(2009, 11, 12, 11, 14)},
             
             {"author": "Eliot",
              "title": "MongoDB is fun",
              "text": "and pretty easy too!",
              "date": datetime.datetime(2009, 11, 10, 10, 45)}]

result = posts.insert_many(new_posts)
result.inserted_ids

[ObjectId('5f9ab5edb9603eb43d9407ee'), ObjectId('5f9ab5edb9603eb43d9407ef')]

There are a couple of interesting things to note about this example:

+ The result from insert_many() now returns two ObjectId instances, one for each inserted document.
+ new_posts[1] has a different “shape” than the other posts - there is no "tags" field and we’ve added a new field, "title". This is what we mean when we say that MongoDB is __schema-free__.  

## Querying for More Than One Document

To get more than a single document as the result of a query we use the find() method. find() returns a Cursor instance, which allows us to iterate over all matching documents. For example, we can iterate over every document in the posts collection:

In [22]:
for post in posts.find():
    pprint.pprint(post)

{'_id': ObjectId('5f9ab5edb9603eb43d9407ed'),
 'author': 'Mike',
 'date': datetime.datetime(2020, 10, 29, 12, 30, 37, 656000),
 'tags': ['mongodb', 'python', 'pymongo'],
 'text': 'My first blog post!'}
{'_id': ObjectId('5f9ab5edb9603eb43d9407ee'),
 'author': 'Mike',
 'date': datetime.datetime(2009, 11, 12, 11, 14),
 'tags': ['bulk', 'insert'],
 'text': 'Another post!'}
{'_id': ObjectId('5f9ab5edb9603eb43d9407ef'),
 'author': 'Eliot',
 'date': datetime.datetime(2009, 11, 10, 10, 45),
 'text': 'and pretty easy too!',
 'title': 'MongoDB is fun'}


Just like we did with find_one(), we can pass a document to find() to limit the returned results. Here, we get only those documents whose author is “Mike”:

In [23]:
for post in posts.find({"author": "Mike"}):
    pprint.pprint(post)

{'_id': ObjectId('5f9ab5edb9603eb43d9407ed'),
 'author': 'Mike',
 'date': datetime.datetime(2020, 10, 29, 12, 30, 37, 656000),
 'tags': ['mongodb', 'python', 'pymongo'],
 'text': 'My first blog post!'}
{'_id': ObjectId('5f9ab5edb9603eb43d9407ee'),
 'author': 'Mike',
 'date': datetime.datetime(2009, 11, 12, 11, 14),
 'tags': ['bulk', 'insert'],
 'text': 'Another post!'}


## Counting

If we just want to know how many documents match a query we can perform a count_documents() operation instead of a full query. We can get a count of all of the documents in a collection:

In [24]:
posts.count_documents({})

3

In [25]:
posts.count_documents({"author": "Mike"})

2

## Range Queries

MongoDB supports many different types of advanced queries. As an example, lets perform a query where we limit results to posts older than a certain date, but also sort the results by author:  
Here we use the special "$lt" operator to do a range query, and also call sort() to sort the results by author.

In [26]:
d = datetime.datetime(2009, 11, 12, 12)
for post in posts.find({"date": {"$lt": d}}).sort("author"):
    pprint.pprint(post)

{'_id': ObjectId('5f9ab5edb9603eb43d9407ef'),
 'author': 'Eliot',
 'date': datetime.datetime(2009, 11, 10, 10, 45),
 'text': 'and pretty easy too!',
 'title': 'MongoDB is fun'}
{'_id': ObjectId('5f9ab5edb9603eb43d9407ee'),
 'author': 'Mike',
 'date': datetime.datetime(2009, 11, 12, 11, 14),
 'tags': ['bulk', 'insert'],
 'text': 'Another post!'}


## Indexing

Adding indexes can help accelerate certain queries and can also add additional functionality to querying and storing documents. In this example, we’ll demonstrate how to create a unique index on a key that rejects documents whose value for that key already exists in the index.

First, we’ll need to create the index:

In [27]:
result = db.profiles.create_index([('user_id', pymongo.ASCENDING)], unique=True)
sorted(list(db.profiles.index_information()))

['_id_', 'user_id_1']

Notice that we have two indexes now: one is the index on _id that MongoDB creates automatically, and the other is the index on user_id we just created.

Now let’s set up some user profiles:

In [28]:
user_profiles = [{'user_id': 211, 'name': 'Luke'},
                 {'user_id': 212, 'name': 'Ziltoid'}]

result = db.profiles.insert_many(user_profiles)

The index prevents us from inserting a document whose user_id is already in the collection:

In [29]:
new_profile = {'user_id': 213, 'name': 'Drew'}
duplicate_profile = {'user_id': 212, 'name': 'Tommy'}

In [30]:
result = db.profiles.insert_one(new_profile)  # This is fine.

In [31]:
#result = db.profiles.insert_one(duplicate_profile) ## error

## Delete Document / Collection / Database

You can delete a document, a collection or a database in MongoDB, by using the methods:
+ delete_one()
+ delete_many()
+ drop() 
+ drop_database()

### Delete Document(s)

In [32]:
# db = client.test_database
mycol = db["customers"]

mylist = [{ "_id": 1, "name": "John", "address": "Highway 37"},
          { "_id": 2, "name": "Peter", "address": "Lowstreet 27"},
          { "_id": 3, "name": "Amy", "address": "Apple st 652"},
          { "_id": 4, "name": "Hannah", "address": "Mountain 21"},
          { "_id": 5, "name": "Michael", "address": "Valley 345"},
          { "_id": 6, "name": "Sandy", "address": "Ocean blvd 2"},
          { "_id": 7, "name": "Betty", "address": "Green Grass 1"},
          { "_id": 8, "name": "Richard", "address": "Sky st 331"},
          { "_id": 9, "name": "Susan", "address": "One way 98"},
          { "_id": 10, "name": "Vicky", "address": "Yellow Garden 2"},
          { "_id": 11, "name": "Ben", "address": "Park Lane 38"},
          { "_id": 12, "name": "William", "address": "Central st 954"},
          { "_id": 13, "name": "Chuck", "address": "Main Road 989"},
          { "_id": 14, "name": "Viola", "address": "Sideway 1633"}
         ]

customers = mycol.insert_many(mylist)

#print list of the _id values of the inserted documents:
print(customers.inserted_ids)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]


Deleting one

In [33]:
myquery = {"address": "Mountain 21"}
mycol.delete_one(myquery)

Deleting many

In [34]:
myquery = {"address": {"$regex": "^S"}}
deleted = mycol.delete_many(myquery)

print(deleted.deleted_count, " documents deleted.")

2  documents deleted.


Deleting all

In [35]:
deleted = mycol.delete_many({})

### Delete Collection

In [36]:
print(db.list_collection_names())

['customers', 'profiles', 'posts']


In [37]:
posts.drop()

In [38]:
print(db.list_collection_names())

['customers', 'profiles']


### Delete Database

In [39]:
print(client.list_database_names())

['admin', 'config', 'local', 'test_database']


In [40]:
client.drop_database('test_database')

In [41]:
print(client.list_database_names())

['admin', 'config', 'local']
